# Visualization

## Open an example model & simulate

In [ ]:
!pip install -q sme
import sme
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML

In [ ]:
my_model = sme.open_example_model()
sim_results = my_model.simulate(simulation_time=250.0, image_interval=5.0)

## Animation of species concentrations

In [ ]:
species = ["B_cell", "B_out"]

fig, axs = plt.subplots(constrained_layout=True, ncols=len(species), figsize=(9, 4))

# set normalization of each plot to maximum concentration of species over entire simulation
norms = []
for spec, ax in zip(species, axs):
    c_max = np.max([np.max(r.species_concentration[spec]) for r in sim_results])
    norms.append(plt.Normalize(vmin=0, vmax=c_max))

# make a plot with correctly normalized colorbar for each species
for ax, norm in zip(axs, norms):
    im = ax.imshow(np.zeros((1, 1)), norm=norm)
    fig.colorbar(im, ax=ax)

# create a list of plot artists for each timepoint
artists = []
for sim_result in sim_results:
    artist = []
    for spec, ax, norm in zip(species, axs, norms):
        artist.append(
            ax.imshow(
                sim_result.species_concentration[spec][0],
                animated=True,
                norm=norm,
                interpolation=None,
            )
        )
        artist.append(
            ax.text(
                0.5,
                1.01,
                f"{spec}: t = {sim_result.time_point}",
                horizontalalignment="center",
                verticalalignment="bottom",
                transform=ax.transAxes,
            )
        )
    artists.append(artist)

# make an animation from the list of artists
anim = animation.ArtistAnimation(fig, artists, interval=200, blit=True, repeat=False)
plt.close()
HTML(anim.to_html5_video())

## Gray-Scott Model

In [ ]:
# simulate for a few different reaction parameter values & store concentrations of species V
gray_scott = sme.open_example_model("gray-scott")
fs = ["0.03", "0.04", "0.05"]
vs = []
for f in fs:
    gray_scott.parameters["f"].value = f
    sim_results = gray_scott.simulate(
        simulation_time=10000.0,
        image_interval=100,
        simulator_type=sme.SimulatorType.Pixel,
    )
    times = [r.time_point for r in sim_results]
    vs.append([r.species_concentration["V"][0] for r in sim_results])

In [ ]:
fig, axs = plt.subplots(constrained_layout=True, nrows=len(fs), figsize=(9, 16))
for ax, f in zip(axs, fs):
    ax.set_title(f"f = {f}")

artists = []
for i in range(len(vs[0])):
    artist = []
    for ax, v in zip(axs, vs):
        artist.append(ax.imshow(v[i], animated=True, interpolation=None))
    artists.append(artist)

anim = animation.ArtistAnimation(fig, artists, interval=200, blit=True, repeat=False)
plt.close()
HTML(anim.to_html5_video())